<a href="https://colab.research.google.com/github/ATalaveraDev/social-network-extraction-info/blob/main/ERS_Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dependencies

In [1]:
pip install tweepy

Obtener los Estados de Estados Unidos (solo los incorporados)

In [74]:
import requests
from bs4 import BeautifulSoup

URL = "https://www23.statcan.gc.ca/imdb/p3VD.pl?Function=getVD&TVD=53971"
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

states = {}
for row in soup.find_all('tr'):
  tds = row.find_all('td')
  if (len(tds) > 0):
    cells = [d.text for d in tds]
    states[cells[2]] = cells[0]
states

{'AK': 'Alaska',
 'AL': 'Alabama',
 'AR': 'Arkansas',
 'AZ': 'Arizona',
 'CA': 'California',
 'CO': 'Colorado',
 'CT': 'Connecticut',
 'DC': 'District of Columbia',
 'DE': 'Delaware',
 'FL': 'Florida',
 'GA': 'Georgia',
 'HI': 'Hawaii',
 'IA': 'Iowa',
 'ID': 'Idaho',
 'IL': 'Illinois',
 'IN': 'Indiana',
 'KS': 'Kansas',
 'KY': 'Kentucky',
 'LA': 'Louisiana',
 'MA': 'Massachusetts',
 'MD': 'Maryland',
 'ME': 'Maine',
 'MI': 'Michigan',
 'MN': 'Minnesota',
 'MO': 'Missouri',
 'MS': 'Mississippi',
 'MT': 'Montana',
 'NC': 'North Carolina',
 'ND': 'North Dakota',
 'NE': 'Nebraska',
 'NH': 'New Hampshire',
 'NJ': 'New Jersey',
 'NM': 'New Mexico',
 'NV': 'Nevada',
 'NY': 'New York',
 'OH': 'Ohio',
 'OK': 'Oklahoma',
 'OR': 'Oregon',
 'PA': 'Pennsylvania',
 'RI': 'Rhode Island',
 'SC': 'South Carolina',
 'SD': 'South Dakota',
 'TN': 'Tennessee',
 'TX': 'Texas',
 'UT': 'Utah',
 'VA': 'Virginia',
 'VT': 'Vermont',
 'WA': 'Washington',
 'WI': 'Wisconsin',
 'WV': 'West Virginia',
 'WY': 'Wyoming

OAuth authentication

In [3]:
import tweepy

auth = tweepy.OAuthHandler('GJdVKUjoEl31WLHlgIRYxVmYr', 'HVaOvQWoHIkiHZh2yPNb0YMxwR6waHagtoR4hcKURo6nhvGxwL')
auth.set_access_token('1254452545583529991-g0sI2XhrtijaQ5zA3NNPRwCfuSKrCr', '4GLEprsRqsKlHPelKC4YirECbHTQYsJJGYiy0JN2OuRiH')

api = tweepy.API(auth)

Tweets Selection

In [37]:
from tweepy.streaming import StreamListener
from tweepy import Stream
import json
import time

class StdOutListener(StreamListener):
    def on_data(self, data):
      tweet = json.loads(data)
      tweets.append({ 'text': tweet['text'], 'place': tweet['place']['full_name'] })
      return True

    def on_error(self, status):
        print(status)

tweets = []
l = StdOutListener()
stream = Stream(auth, l)
stream.filter(languages=['en'], locations=[-171.791110603, 18.91619, -66.96466, 71.3577635769], is_async=True)
time.sleep(10)
stream.disconnect()


DONE!
[{'text': '@barbiebayxx @NaijaEverything We from Africa it ain’t née plus I like younger girls 🌚🌚 she looks sweet', 'place': 'Georgia, USA'}, {'text': '@lonliestwolf Gooood....goooood https://t.co/QdQKaWl8xK', 'place': 'Columbus, OH'}, {'text': 'it’d be a lie to say that shit didn’t hurt my feelings.', 'place': 'Dallas, TX'}, {'text': 'I can’t complain...🤷🏾\u200d♂️... it was all 4 the taking 🙇🏾\u200d♂️', 'place': 'Oakland, CA'}, {'text': 'Flat Rock / Coon Creek: Running https://t.co/XnDGONy515', 'place': 'Alabama, USA'}, {'text': 'Don’t let No one get comfortable Disrespecting You!!!', 'place': 'Scio, MI'}, {'text': '@Thomas1774Paine Just like he was a Russian puppet right Eric? \nHow’s that Russian Collusion fairytale going? Have… https://t.co/38iy7V6MgC', 'place': 'Russellton, PA'}, {'text': '@Oluomachi_u I believe in you!😭', 'place': 'Chicago, IL'}, {'text': '@iTS_MeMadiO You work at the 🏥', 'place': 'Dallas, TX'}]


Sentiment Processor

In [95]:
feelings = open('AFINN-111.txt', 'r')

def initFeelingsDictionary():
  dict = {}
  for feeling in feelings:
    feelingSplit = feeling.split('\\n')
    feel = feelingSplit[0].replace('\n', '').split('\t')
    dict[feel[0]] = feel[1]
  return dict

affinn = initFeelingsDictionary()

testData = [{'text': '@barbiebayxx @NaijaEverything We from Africa it ain’t née plus I like younger girls 🌚🌚 she looks sweet', 'place': 'Georgia, USA'}, {'text': '@lonliestwolf Gooood....goooood https://t.co/QdQKaWl8xK', 'place': 'Columbus, OH'}, {'text': 'it’d be a lie to say that shit didn’t hurt my feelings.', 'place': 'Dallas, TX'}, {'text': 'I can’t complain...🤷🏾\u200d♂️... it was all 4 the taking 🙇🏾\u200d♂️', 'place': 'Oakland, CA'}, {'text': 'Flat Rock / Coon Creek: Running https://t.co/XnDGONy515', 'place': 'Alabama, USA'}, {'text': 'Don’t let No one get comfortable Disrespecting You!!!', 'place': 'Scio, MI'}, {'text': '@Thomas1774Paine Just like he was a Russian puppet right Eric? \nHow’s that Russian Collusion fairytale going? Have… https://t.co/38iy7V6MgC', 'place': 'Russellton, PA'}, {'text': '@Oluomachi_u I believe in you!😭', 'place': 'Chicago, IL'}, {'text': '@iTS_MeMadiO You work at the 🏥', 'place': 'Dallas, TX'}]

for tweet in testData:
  code = tweet['place'].split(', ')[1]
  
  if code in states:
    happinessIndex = 0
    for word in tweet['text'].split():
      if word.lower() in affinn:
        happinessIndex += int(affinn[word.lower()])
    print({ 'tweet': tweet['place'].split(',')[1], 'happiness': happinessIndex })

{'tweet': ' OH', 'happiness': 0}
{'tweet': ' TX', 'happiness': -6}
{'tweet': ' CA', 'happiness': 0}
{'tweet': ' MI', 'happiness': 1}
{'tweet': ' PA', 'happiness': 2}
{'tweet': ' IL', 'happiness': 0}
{'tweet': ' TX', 'happiness': 0}


Map